**Для начала повторяем код от третьего урока.**

In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://spb.hh.ru/search/vacancy?text=%D0%93%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&excluded_text='


In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

In [4]:
def parse_hh(url_page, headers, result=[]):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cсылка: %s'%response.url)

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary


In [5]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cсылка: https://spb.hh.ru/search/vacancy?text=%D0%93%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&excluded_text=
Cсылка: https://spb.hh.ru/search/vacancy?text=%D0%93%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&excluded_text=&page=1&hhtmFrom=vacancy_search_list
Cсылка: https://spb.hh.ru/search/vacancy?text=%D0%93%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&excluded_text=&page=2&hhtmFrom=vacancy_search_list
Cсылка: https://spb.hh.ru/search/vacancy?text=%D0%93%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&excluded_text=&page=3&hhtmFrom=vacancy_search_list
Cсылка: https://spb.hh.ru/search/vacancy?text=%D0%93%D1%

In [6]:
len(result)

235

In [7]:
result

[{'vacancy_name': 'Грузчик / Разнорабочий 300 в час',
  'vacancy_salary': '30\u202f000 – 70\u202f000 руб.',
  'min_salary': 30000,
  'max_salary': 70000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/77718883?from=vacancy_search_list&query=%D0%93%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Грузчик',
  'vacancy_salary': 'от 40\u202f000 руб.',
  'min_salary': 40000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/76914898?from=vacancy_search_list&query=%D0%93%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Грузчик',
  'vacancy_salary': 'от 45\u202f000 руб.',
  'min_salary': 45000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://spb.hh.ru/vacancy/76698999?from=vacancy_search_list&query=%D0%93%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Грузчик-комплектовщик',
  

**Далее работаем с MongoDB.**

In [8]:
!apt install mongodb > log

In [9]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [10]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from pymongo import MongoClient

In [12]:
client = MongoClient()

In [13]:
db = client.vacancies_python_hh

In [14]:
collection_vacancies_hh_ru = db.hh_ru

**Проверим заполняемость базы.**

In [15]:
def cheak_and_save_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_hh_ru.find({'vacancy_link': vacancy['vacancy_link']}))):
      collection_vacancies_hh_ru.insert_one(vacancy)

In [16]:
cheak_and_save_vacancies_in_db(result)

In [17]:
result_find = list(collection_vacancies_hh_ru.find())

In [18]:
len(result_find)

235